### Baseline модель для определения именованных сущностей по кейсу от Rutube.
Поскольку нам нужно распознать нестандартные NER, можно воспользоваться помощью языковых моделей, в данном случае - Bert.
Данные вы уже получили  - это разметка, сделанная на Толоке, она не идеальна, но это часть практической задачи, с которой можно столкнуться в реальности. 

Небольшое введение в NER https://habr.com/ru/companies/contentai/articles/449514/

In [1]:
# считаем данные
import pandas as pd
data = pd.read_csv("ner_data_train.csv")

In [2]:
data.head(5)

video_info  \
0  <НАЗВАНИЕ:> Агент 117: Из Африки с любовью — Р...   
1  <НАЗВАНИЕ:> Коленвал Инфинити Ку икс 56= 5.6 V...   
2  <НАЗВАНИЕ:> ВЫЗОВ ДЕМОНА = Вызвал Серого Челов...   
3  <НАЗВАНИЕ:> Довоенная немецкая кирха в Калинин...   
4  <НАЗВАНИЕ:> "Спартаку" помогли судьи? Локомоти...   

                                            entities  
0  {"label":"локация"\,"offset":26\,"length":6\,"...  
1  {"label":"организация"\,"offset":196\,"length"...  
2  {"label":"название проекта"\,"offset":12\,"len...  
3  {"label":"не найдено"\,"offset":162\,"length":...  
4  {"label":"команда"\,"offset":13\,"length":8\,"...

In [3]:
# данные спарсены с Толоки, поэтому могут иметь проблемы с символами и их нужно избежать, 
# удалить лишние '\' например, преобразовать из str в список dict-ов
import json
df = data.copy()
df['entities'] = df['entities'].apply(lambda l: l.replace('\,', ',')if isinstance(l, str) else l)
df['entities'] = df['entities'].apply(lambda l: l.replace('\\\\', '\\')if isinstance(l, str) else l)
df['entities'] = df['entities'].apply(lambda l: '[' + l + ']'if isinstance(l, str) else l)
df['entities'] = df['entities'].apply(lambda l: json.loads(l)if isinstance(l, str) else l)

In [4]:
df.head(3)

video_info  \
0  <НАЗВАНИЕ:> Агент 117: Из Африки с любовью — Р...   
1  <НАЗВАНИЕ:> Коленвал Инфинити Ку икс 56= 5.6 V...   
2  <НАЗВАНИЕ:> ВЫЗОВ ДЕМОНА = Вызвал Серого Челов...   

                                            entities  
0  [{'label': 'локация', 'offset': 26, 'length': ...  
1  [{'label': 'организация', 'offset': 196, 'leng...  
2  [{'label': 'название проекта', 'offset': 12, '...

#### Оригинал туториала на медицинских данных можно посмотреть тут https://gist.github.com/avidale/cacf235aebeaaf4c578389e1146c3c57

In [5]:
# Теперь из наших данных нам нужно извлечь для каждого слова (токена) его тег (label) из разметки, чтобы потом предать в модель классификации токенов
from razdel import tokenize

def extract_labels(item):
    
    # воспользуемся удобным токенайзером из библиотеки razdel, 
    # она помимо разбиения на слова, сохраняет важные для нас числа - начало и конец слова в токенах
    
    raw_toks = list(tokenize(item['video_info']))
    words = [tok.text for tok in raw_toks]
    # присвоим для начала каждому слову тег 'О' - тег, означающий отсутствие NER-а
    word_labels = ['O'] * len(raw_toks)
    char2word = [None] * len(item['video_info'])
    # так как NER можем состаять из нескольких слов, то нам нужно сохранить эту инфорцию
    for i, word in enumerate(raw_toks):
        char2word[word.start:word.stop] = [i] * len(word.text)

    labels = item['entities']
    if isinstance(labels, dict):
        labels = [labels]
    if labels is not None:
        for e in labels:
            if e['label'] != 'не найдено':
                e_words = sorted({idx for idx in char2word[e['offset']:e['offset']+e['length']] if idx is not None})
                if e_words:
                    word_labels[e_words[0]] = 'B-' + e['label']
                    for idx in e_words[1:]:
                        word_labels[idx] = 'I-' + e['label']
                else:
                    continue
            else:
                continue
        return {'tokens': words, 'tags': word_labels}
    else: return {'tokens': words, 'tags': word_labels}

In [6]:
print(extract_labels(df.iloc[0]))

{'tokens': ['<', 'НАЗВАНИЕ', ':', '>', 'Агент', '117', ':', 'Из', 'Африки', 'с', 'любовью', '—', 'Русский', 'тизер', '=', 'трейлер', '(', '2021', ')', '<', 'ОПИСАНИЕ', ':', '>', 'Лучший', 'Telegram', 'канал', 'о', 'кино', '<', 'LINK', '>', 'Сотрудничество', '<', 'LINK', '>', 'Дата', 'выхода', '26', 'августа', '2021', 'Оригинальное', 'название', ':', 'OSS', '117', ':', 'Alerte', 'rouge', 'en', 'Afrique', 'noire', 'Страна', ':', 'Франция', 'Режиссер', ':', 'Николя', 'Бедос', 'Жанр', ':', 'боевик', ',', 'комедия', 'В', 'главных', 'ролях', ':', 'Жан', 'Дюжарден', ',', 'Пьер', 'Нинэ', ',', 'Мелоди', 'Каста', ',', 'Наташа', 'Линдинжер', ',', 'Владимир', 'Иорданов', ',', 'Фату', 'Н', '’', 'Диайе', ',', 'Пол', 'Уайт', 'Мир', 'изменился', '.', 'Он', 'нет', '.', 'Судьба', 'заносит', 'легендарного', 'Агента', '117', 'в', 'Африку', ',', 'где', 'горячее', 'пустыни', 'только', 'женщины', '.', 'Вооруженный', 'неиссякаемой', 'уверенностью', 'в', 'себе', 'и', 'убийственной', 'харизмой', ',', 'он', 'мож

### Обработаем датасет и разобьем на трейн и тест

In [7]:
from sklearn.model_selection import train_test_split
ner_data = [extract_labels(item) for i, item in df.iterrows()]
ner_train, ner_test = train_test_split(ner_data, test_size=0.2, random_state=1)

In [8]:
import pandas as pd
pd.options.display.max_colwidth = 300
pd.DataFrame(ner_train).sample(3)

tokens  \
4812  [<, НАЗВАНИЕ, :, >, Система, построения, бизнес, с, WorldVentures, от, Алены, Антоненко, <, ОПИСАНИЕ, :, >, До, официального, открытия, компании, WorldVentures, в, России, осталось, совсем, немного, времени, ., Применяя, простую, и, дуплицируемую, систему, построения, бизнеса, ,, можно, за, коро...   
2915  [<, НАЗВАНИЕ, :, >, Хрустальное, счастье, 1, =, 4, серия, (, Мелодрама, 2021, ), обзор, ,, дата, выхода, <, ОПИСАНИЕ, :, >, Премьера, 2021, ", Хрустальное, счастье, ", 1, 4, серии, обзор, ,, дата, выхода, на, Россия, 1, ., Главные, роли, исполнят, :, Мария, Куликова, ,, Юрий, Батурин, ,, Дан, Ро...   
4190  [<, НАЗВАНИЕ, :, >, Этой, Ночью, ГОЛОВКИН, побил, все, МИРОВЫЕ, РЕКОРДЫ, !, На, Это, было, Страшно, Смотреть, !, <, ОПИСАНИЕ, :, >, Всем, привет, ,, дорогие, друзья, !, Вы, на, канале, Безумный, Рассказчик, 2, ., СМОТРИ, ТАКЖЕ, :, Мэнни, Пакьяо, Уничтожает, Крупных, Боксеров, <, LINK, >, Этим, У...   

                                                                                                                                                                                                                                                                                                             tags  
4812                                                                       [O, O, O, O, O, O, O, O, B-персона, O, B-организация, I-организация, O, O, O, O, O, O, O, O, B-персона, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-персона, O, O, B-персона, O, O, O, O, O]  
2915  [O, O, O, O, B-название проекта, I-название проекта, B-серия, I-серия, I-серия, I-серия, O, O, B-Дата, O, O, O, O, O, O, O, O, O, O, B-Дата, O, B-название проекта, I-название проекта, O, B-серия, I-серия, I-серия, O, O, O, O, O, B-локация, O, O, O, O, O, O, B-персона, I-персона, O, B-персона, I-...  
4190  [O, O, O, O, O, O, B-персона, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-название проекта, I-название проекта, I-название проекта, O, O, O, O, B-персона, I-персона, O, O, O, O, O, O, O, O, B-персона, O, O, O, O, O, O, O, B-персона, I-персона, O, O, O, O, O, O, O, O...

#### Посмотрим на получившиеся теги
Подробнее почитать про BIO теги можно тут https://datascience.stackexchange.com/questions/63399/what-is-bio-tags-for-creating-custom-ner-named-entity-recognization

In [9]:
label_list = sorted({label for item in ner_train for label in item['tags']})
if 'O' in label_list:
    label_list.remove('O')
    label_list = ['O'] + label_list
label_list

['O',
 'B-Дата',
 'B-бренд',
 'B-вид спорта',
 'B-видеоигра',
 'B-команда',
 'B-лига',
 'B-локация',
 'B-модель',
 'B-название проекта',
 'B-организация',
 'B-персона',
 'B-сезон',
 'B-серия',
 'I-Дата',
 'I-бренд',
 'I-вид спорта',
 'I-видеоигра',
 'I-команда',
 'I-лига',
 'I-локация',
 'I-модель',
 'I-название проекта',
 'I-организация',
 'I-персона',
 'I-сезон',
 'I-серия']

In [10]:
from datasets import Dataset, DatasetDict

In [11]:
ner_data = DatasetDict({
    'train': Dataset.from_pandas(pd.DataFrame(ner_train)),
    'test': Dataset.from_pandas(pd.DataFrame(ner_test))
})
ner_data

DatasetDict({
    train: Dataset({
        features: ['tokens', 'tags'],
        num_rows: 5137
    })
    test: Dataset({
        features: ['tokens', 'tags'],
        num_rows: 1285
    })
})

### Запустим модель RuBert-tiny - классический Bert, поверх которого навешен слой классификации токенов.

In [12]:
from transformers import AutoTokenizer 
from datasets import load_dataset, load_metric

model_checkpoint = "cointegrated/rubert-tiny"
batch_size = 16

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, device='cpu')

In [13]:
example = ner_train[5]
print(example["tokens"])

['<', 'НАЗВАНИЕ', ':', '>', 'Московский', 'международный', 'фестиваль', 'мира', '=', '89', 'и', 'Стас', 'Намин', '.', '«', 'Главный', 'день', '»', '<', 'ОПИСАНИЕ', ':', '>', '12', 'и', '13', 'июня', '1989', 'года', 'на', 'Центральном', 'стадионе', 'имени', 'В', '.', 'И', '.', 'Ленина', 'состоялся', 'первый', 'международный', 'рок', 'фестиваль', '.', 'Концерт', 'собрал', 'свыше', '100', 'тысяч', 'зрителей', '.', 'Советская', 'молодежь', 'тогда', 'увидела', 'и', 'услышала', 'признанных', 'мировых', 'кумиров', ':', 'Scorpions', ',', 'Bon', 'Jovi', ',', 'Ozzy', 'Osbourne', ',', 'Cinderella', ',', 'Motley', 'Crue', '.', 'Приезд', 'западных', 'звезд', 'казался', 'ожившей', 'фантастикой', '.', 'Фестиваль', 'стал', 'не', 'только', 'музыкальной', ',', 'но', 'и', 'важной', 'политической', 'акцией', '.', 'Вся', 'мировая', 'пресса', 'писала', 'о', 'том', ',', 'что', 'СССР', 'и', 'США', ',', 'долгое', 'время', 'находившиеся', 'в', 'состоянии', '"', 'холодной', 'войны', '"', ',', 'наконец', 'то', 'с

In [14]:
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
print(tokens)

['[CLS]', '<', 'Н', '##АЗ', '##В', '##АН', '##И', '##Е', ':', '>', 'Московский', 'между', '##народный', 'фестиваль', 'мира', '=', '89', 'и', 'С', '##тас', 'На', '##мин', '.', '«', 'Главный', 'день', '»', '<', 'О', '##П', '##И', '##С', '##АН', '##И', '##Е', ':', '>', '12', 'и', '13', 'июня', '1989', 'года', 'на', 'Центр', '##альном', 'стадион', '##е', 'имени', 'В', '.', 'И', '.', 'Ленина', 'состоялся', 'первый', 'между', '##народный', 'рок', 'фестиваль', '.', 'К', '##он', '##цер', '##т', 'со', '##брал', 'свыше', '100', 'тысяч', 'зрителей', '.', 'Советская', 'м', '##оло', '##де', '##ж', '##ь', 'тогда', 'у', '##вид', '##ела', 'и', 'у', '##сл', '##ыш', '##ала', 'признан', '##ных', 'мир', '##овых', 'к', '##уми', '##ров', ':', 'Sc', '##orp', '##ions', ',', 'Bon', 'Jovi', ',', 'Oz', '##zy', 'Os', '##bour', '##ne', ',', 'Ci', '##nder', '##ella', ',', 'Mot', '##ley', 'C', '##rue', '.', 'При', '##езд', 'запад', '##ных', 'зв', '##езд', 'каз', '##ался', 'о', '##жив', '##шей', 'ф', '##ант', '##аст'

In [15]:
len(example["tags"]), len(tokenized_input["input_ids"])

(170, 274)

In [16]:
print(tokenized_input.word_ids())

[None, 0, 1, 1, 1, 1, 1, 1, 2, 3, 4, 5, 5, 6, 7, 8, 9, 10, 11, 11, 12, 12, 13, 14, 15, 16, 17, 18, 19, 19, 19, 19, 19, 19, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 29, 30, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 39, 40, 41, 42, 43, 43, 43, 43, 44, 44, 45, 46, 47, 48, 49, 50, 51, 51, 51, 51, 51, 52, 53, 53, 53, 54, 55, 55, 55, 55, 56, 56, 57, 57, 58, 58, 58, 59, 60, 60, 60, 61, 62, 63, 64, 65, 65, 66, 66, 66, 67, 68, 68, 68, 69, 70, 70, 71, 71, 72, 73, 73, 74, 74, 75, 75, 76, 76, 77, 77, 77, 78, 78, 78, 78, 79, 80, 80, 80, 81, 82, 83, 84, 85, 86, 87, 88, 88, 89, 90, 90, 90, 91, 92, 92, 93, 93, 94, 94, 95, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 104, 105, 106, 106, 106, 107, 108, 109, 110, 110, 110, 111, 112, 113, 114, 115, 116, 117, 117, 117, 118, 118, 118, 118, 119, 120, 121, 121, 122, 123, 124, 124, 125, 125, 126, 126, 127, 127, 127, 128, 128, 129, 130, 130, 130, 130, 131, 132, 132, 133, 133, 134, 135, 136, 137, 137, 137, 138, 139, 139, 139, 140, 141, 142, 143, 144, 144, 144, 

In [17]:
word_ids = tokenized_input.word_ids()
aligned_labels = [-100 if i is None else example["tags"][i] for i in word_ids]
print(len(aligned_labels), len(tokenized_input["input_ids"]))

274 274


#### У Bert свой собсвенный токенайзер, который разбивает слова на мелкие токены, поэтому нам нужно корректно сопоставить токены и соответсвующие им неры.

In [18]:
def tokenize_and_align_labels(examples, label_all_tokens=True):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples['tags']):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        label_ids = [label_list.index(idx) if isinstance(idx, str) else idx for idx in label_ids]

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [19]:
tokenize_and_align_labels(ner_data['train'][1:2])

{'input_ids': [[2, 32, 293, 16218, 8117, 20795, 8759, 14386, 30, 34, 16992, 12449, 19314, 9147, 705, 19062, 1211, 2179, 15349, 603, 32, 294, 3932, 8759, 3330, 20795, 8759, 14386, 30, 34, 294, 18147, 5972, 2386, 1928, 733, 22970, 613, 17565, 29484, 15479, 12122, 18398, 13334, 1, 16992, 12449, 19314, 9147, 705, 19062, 1211, 2179, 15349, 603, 18, 282, 3200, 860, 20727, 12029, 4375, 29463, 2013, 2262, 18, 294, 15735, 29137, 320, 18029, 18398, 24952, 753, 705, 11316, 4307, 329, 22067, 12427, 283, 4297, 292, 29172, 3330, 1928, 705, 13790, 294, 22797, 1768, 10203, 761, 13951, 3243, 2535, 2641, 18, 3]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [20]:
tokenized_datasets = ner_data.map(tokenize_and_align_labels, batched=True)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Map:   0%|          | 0/5137 [00:00<?, ? examples/s]

Map:   0%|          | 0/1285 [00:00<?, ? examples/s]

#### Сохраняем словарик соотвествия тега и его индекса внутри модели

In [21]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))
model.config.id2label = dict(enumerate(label_list))
model.config.label2id = {v: k for k, v in model.config.id2label.items()}

Some weights of the model checkpoint at cointegrated/rubert-tiny were not used when initializing BertForTokenClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at cointegrated/rubert-tiny and are newly

In [22]:
# Специальный объект для удобного формирования батчей
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

### В качестве метрик возьмем precision, recall, accuracy, для этого воспользуемся специализированной под Ner задачу библиотеку seqeval

In [23]:
metric = load_metric("seqeval")

/var/folders/pw/qs4ys1kd68sbr86_46f_tgz80000gp/T/ipykernel_20329/152412463.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("seqeval")


In [24]:
example = ner_train[4]
labels = example['tags']
metric.compute(predictions=[labels], references=[labels])

{'бренд': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'локация': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'overall_precision': 1.0,
 'overall_recall': 1.0,
 'overall_f1': 1.0,
 'overall_accuracy': 1.0}

In [25]:
args = TrainingArguments(
    "ner",
    evaluation_strategy = "epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=20,
    weight_decay=0.01,
    save_strategy='no',
    report_to='none',
)

In [26]:
# что мы видим без дообучения модели
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels, zero_division=0)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.evaluate()

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


RuntimeError: Placeholder storage has not been allocated on MPS device!

In [55]:
pip install torch torchvision torchaudio

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
  Obtaining dependency information for torchaudio from https://files.pythonhosted.org/packages/9e/e2/812b1a2a5ec34fe7c0a5cf11b4e62709172fd71783cbc8b7875e3051e8a5/torchaudio-2.1.0-cp39-cp39-macosx_11_0_arm64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 43.9 kB/s eta 0:00:00m eta 0:00:010:00:01m

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import logging
from transformers.trainer import logger as noisy_logger
noisy_logger.setLevel(logging.WARNING)

In [ ]:
# Для дообучения берта можно эксперементировать с заморозкой/разморозкой разных слоев, здесь мы оставим все слои размороженными 
# Для быстроты обучения можно заморозить всю бертовую часть, кроме классификатора, но тогда качесвто будет похуже
for param in model.parameters():
    param.requires_grad = True

In [ ]:
args = TrainingArguments(
    "ner",
    evaluation_strategy = "epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=20,
    weight_decay=0.01,
    save_strategy='no',
    report_to='none',
)

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate()

In [ ]:
# Посчитаем метрики на отложенном датасете

predictions, labels, _ = trainer.predict(tokenized_datasets["test"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

In [ ]:
from sklearn.metrics import confusion_matrix
import pandas as pd

In [ ]:
cm = pd.DataFrame(
    confusion_matrix(sum(true_labels, []), sum(true_predictions, []), labels=label_list),
    index=label_list,
    columns=label_list
)
cm

In [ ]:
model.save_pretrained('ner_bert.bin')
tokenizer.save_pretrained('ner_bert.bin')

### Посмотрим на результаты

In [ ]:
# text = ' '.join(ner_train[25]['tokens'])
text = ner_train[25]['tokens']

In [28]:
import torch
from transformers import pipeline

pipe = pipeline(model=model, tokenizer=tokenizer, task='ner', aggregation_strategy='average', device='cpu')

def predict_ner(text, tokenizer, model, pipe, verbose=True):
    tokens = tokenizer(text, truncation=True, is_split_into_words=True, return_tensors='pt')
    tokens = {k: v.to(model.device) for k, v in tokens.items()}
    
    with torch.no_grad():
        pred = model(**tokens)
    # print(pred.logits.shape)
    indices = pred.logits.argmax(dim=-1)[0].cpu().numpy()
    token_text = tokenizer.convert_ids_to_tokens(tokens['input_ids'][0])
    labels = []
    for t, idx in zip(token_text, indices):
        if '##' not in t:
            labels.append(label_list[idx])
        if verbose:    
            print(f'{t:15s} {label_list[idx]:10s}')
    return text, pipe(text), labels

In [ ]:
predict_ner(text, tokenizer, model, pipe)

### Тестового датасета у вас пока нет, по которому будет считаться метрика на лидерборде, но прогоним для примера через нашу отложенную выборку, чтобы понять формат выходных данных.
ВАЖНО: в тестовом датасете у вас будет тест в том же формате, что он был в трейне 'video_info', в финальном сабмишене эту колонку и индексы менять нельзя, нужно будет только заполнить колонку 'entities_prediction'

In [ ]:
from tqdm.notebook import tqdm

submission = pd.DataFrame(columns=[['video_info', 'entities_prediction']])
submission['entities_prediction'] = submission['entities_prediction'].astype('object')
def sample_submission(text, tokenizer, model, pipe, submission):
    for i, elem in tqdm(enumerate(ner_test)):
        _, _, labels = predict_ner(elem['tokens'], tokenizer, model, pipe, verbose=False)
        submission.loc[i, 'video_info'] = elem

        submission.loc[i, 'entities_prediction'] = [[label] for label in labels]
    return submission

In [ ]:
result = sample_submission(text, tokenizer, model, pipe, submission)

In [ ]:
result

In [ ]:
len(ner_test)